In [12]:
import numpy as np

In [13]:
import pandas as pd

In [14]:
data_types = {prefix+str(i)+"_Flag":"category" 
              for prefix in ["Temp", "SM"] for i in range(1,7)}

In [15]:
with open("00_data.csv", "r") as f:
    print(f.readline())
    print(f.readline())

"Time" "Box" "Date.Time" "SM1" "SM1_Flag" "Temp1" "Temp1_Flag" "SM2" "SM2_Flag" "Temp2" "Temp2_Flag" "SM3" "SM3_Flag" "Temp3" "Temp3_Flag" "SM4" "SM4_Flag" "Temp4" "Temp4_Flag" "SM5" "SM5_Flag" "Temp5" "Temp5_Flag" "SM6" "SM6_Flag" "Temp6" "Temp6_Flag" "BattV" "SM1_Depth" "SM2_Depth" "SM3_Depth" "SM4_Depth" "SM5_Depth" "SM6_Depth"

"01.2" 2012-07-19 17:45:53 1 "2012-07-19 17:45:53" -2.824 "Auto:Range" 18.258 "OK" -5.9817 "Auto:Range" 18.2232 "OK" -6.4765 "Auto:Range" 18.7101 "OK" -4.5788 "Auto:Range" 18.1189 "OK" -9.5875 "Auto:Range" 18.9884 "OK" -8.584 "Auto:Range" 18.3623 "OK" 3599 0.05 0.05 0.05 0.05 0.05 0.05



In [46]:
data_types = {prefix+str(i)+"_Flag":"category" 
              for prefix in ["Temp", "SM"] for i in range(1,7)}
data_types.update(Box="uint8")
data = pd.read_csv("00_data.csv", delim_whitespace=True,
                  dtype=data_types,
                   header=0,
                   names=[ "idx",
                           "date",
                           "Time",
                           "Box", "Date.Time",
                           "SM1",
                           "SM1_Flag",
                           "Temp1",
                           "Temp1_Flag",
                           "SM2",
                           "SM2_Flag",
                           "Temp2",
                           "Temp2_Flag",
                           "SM3",
                           "SM3_Flag",
                           "Temp3",
                           "Temp3_Flag",
                           "SM4",
                           "SM4_Flag" ,
                           "Temp4",
                           "Temp4_Flag",
                           "SM5" ,
                           "SM5_Flag" ,
                           "Temp5" ,
                           "Temp5_Flag",
                           "SM6" ,
                           "SM6_Flag" ,
                           "Temp6" ,
                           "Temp6_Flag" ,
                           "BattV" ,
                           "SM1_Depth" ,
                           "SM2_Depth",
                           "SM3_Depth" ,
                           "SM4_Depth",
                           "SM5_Depth",
                           "SM6_Depth"], parse_dates=[4])

# time is in hours now, starting at 2010
time_deltas = (data["Date.Time"] - pd.Timestamp("2010-01-01 00:00")).astype('timedelta64[h]')
data["norm_time"] = time_deltas

interesting = ["Box", "Date.Time", "norm_time"] 
base_and_flags = [f"{prefix}{i}{flag}"
                            for prefix in ["Temp", "SM"] 
                            for i in range(1,7)
                            for flag in ["_Flag", ""]] 
sm_depths = [f"SM{i}_Depth" for i in range(1,7)]

# selecting all the interesting columns and renaming them a bit
sel_data = data[interesting + base_and_flags + sm_depths]


In [49]:
sel_data.rename({**{"Box":"box", "Date.Time":"datetime"},
                 **{prefix+str(i)+"_Flag":prefix+str(i)+"_label"
               for prefix in ["Temp", "SM"] for i in range(1,7)}}, axis=1, inplace=True)

In [43]:
sel_data.head()

,box,norm_time,Temp1_label,Temp1,Temp2_label,Temp2,Temp3_label,Temp3,Temp4_label,Temp4,...,SM5_label,SM5,SM6_label,SM6,SM1_Depth,SM2_Depth,SM3_Depth,SM4_Depth,SM5_Depth,SM6_Depth
0,1,22337.0,OK,18.2580,OK,18.2232,OK,18.7101,OK,18.1189,...,Auto:Range,-9.5875,Auto:Range,-8.5840,0.05,0.05,0.05,0.05,0.05,0.05
1,2,6602.0,OK,5.0416,OK,-15.1656,OK,-16.5220,OK,5.3894,...,OK,21.9099,OK,27.5393,0.10,0.20,0.30,0.40,0.50,0.60
2,2,6603.0,OK,8.4848,OK,7.7544,OK,7.1980,OK,7.0588,...,OK,20.9133,Auto:Spike,25.7199,0.10,0.20,0.30,0.40,0.50,0.60
3,2,6677.0,OK,4.9373,OK,-15.1656,OK,-16.5568,OK,5.4937,...,OK,21.4961,OK,27.5369,0.10,0.20,0.30,0.40,0.50,0.60
4,2,6772.0,OK,5.0764,OK,-15.1656,OK,-16.6263,OK,5.2851,...,OK,21.5473,OK,27.5418,0.10,0.20,0.30,0.40,0.50,0.60


In [50]:
box_data = pd.read_csv("00_coordinates_start_end.csv", parse_dates=[2, 3])

box_data["box"] = np.arange(1, 41)

good_time = pd.Timestamp("2014-03"), pd.Timestamp("2015-10")

good_boxes = (box_data.Start < good_time[0]) & ((box_data.End > good_time[1]) | box_data.End.isna())

In [51]:
sel_data = sel_data[(sel_data.box.isin(box_data.box[good_boxes])) & 
                    (sel_data.datetime > good_time[0]) &
                    (sel_data.datetime < good_time[1]) 
                   ]

In [52]:
sel_data.reset_index(inplace=True)
sel_data.to_feather("birdie.feather")